In [1]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time
from skimage import measure

In [2]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty

    # 在这个例子中 cells是(64,64,64)
    a,b,c = cells.shape
    # 遍历每一个cell
    for x in range(a-1):
        for y in range(b-1):
            for z in range(c-1):
                case_nums = getCaseNum(x,y,z,thres,cells)
                for case_num in case_nums:
                    if case_num == -1:
                        break
                    corner1 = (x+CaseNum2EdgeOffset[case_num][0],y+CaseNum2EdgeOffset[case_num][1],z+CaseNum2EdgeOffset[case_num][2])
                    corner2 = (x+CaseNum2EdgeOffset[case_num][3],y+CaseNum2EdgeOffset[case_num][4],z+CaseNum2EdgeOffset[case_num][5])
                    # 插值
                    val1 = cells[corner1]
                    val2 = cells[corner2]

                    interp = np.zeros((3,), dtype=float)

                    mu = (thres - val1) / (val2 - val1)
                    interp[0] = corner1[0] + mu * (corner2[0] - corner1[0])
                    interp[1] = corner1[1] + mu * (corner2[1] - corner1[1])
                    interp[2] = corner1[2] + mu * (corner2[2] - corner1[2])
                    
                    # array不可hash tuple可hash
                    if tuple(interp) in vertex_array:
                        face_array.append(vertex_array[tuple(interp)])

                    else:
                        l = len(vertex_array)
                        vertex_array[tuple(interp)] = l
                        face_array.append(l)
                        
    face_array = np.array(face_array).reshape((-1,3))
                    

    # -------------------TODO------------------ 
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    #vertex_array = list(vertex_array.keys())
    vertex_array = [key for key, value in sorted(vertex_array.items(), key=lambda item: item[1])]
    return np.array(vertex_array), np.array(face_array)

In [3]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
2.7694389820098877 s

Time taken by algorithm
----------------------------------------
2.8282101154327393 s


In [3]:
# 执行库函数Marching Cubes算法用于debug时比对
data = np.load(os.path.join('data','bob_cell.npy'))
vertices, faces, _, _ = measure.marching_cubes(data, level=0)

mesh = trimesh.Trimesh(vertices, faces)

# Export the mesh as an OBJ file
mesh.export('output2.obj', file_type='obj')

'# https://github.com/mikedh/trimesh\nv 0.94745386 37.00000000 31.00000000\nv 1.00000000 36.44673538 31.00000000\nv 1.00000000 37.00000000 30.42052841\nv 0.96528059 37.00000000 32.00000000\nv 1.00000000 36.62054062 32.00000000\nv 1.00000000 37.00000000 32.32704926\nv 1.00000000 37.20889664 31.00000000\nv 1.00000000 37.14471817 32.00000000\nv 1.98987174 34.00000000 32.00000000\nv 2.00000000 33.98643494 32.00000000\nv 2.00000000 34.00000000 31.53816414\nv 2.00000000 34.00000000 32.11009216\nv 1.71078610 35.00000000 29.00000000\nv 2.00000000 34.54009628 29.00000000\nv 2.00000000 35.00000000 28.24238396\nv 1.49663985 35.00000000 30.00000000\nv 2.00000000 34.19188309 30.00000000\nv 1.37943244 35.00000000 31.00000000\nv 2.00000000 34.01752090 31.00000000\nv 1.37449503 35.00000000 32.00000000\nv 2.00000000 34.12137985 33.00000000\nv 1.46821499 35.00000000 33.00000000\nv 1.66747522 35.00000000 34.00000000\nv 2.00000000 34.44973755 34.00000000\nv 2.00000000 35.00000000 34.93639755\nv 1.69238853

In [ ]:
import open3d as o3d

# 读取.obj文件
mesh = o3d.io.read_triangle_mesh("/Users/qiudi/Downloads/03_assignment/results/bob.obj")

# 可视化.obj文件
o3d.visualization.draw_geometries([mesh])
